In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 31.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForQuestionAnswering,
                          Trainer, TrainingArguments, default_data_collator, pipeline, set_seed)
import evaluate
import random
import numpy as np
import torch

In [3]:
raw_dataset = load_dataset('squad')
raw_train   = raw_dataset['train']
raw_eval    = raw_dataset['validation']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
seed = random.randrange(2**32)
print(f"🔢 Using random seed: {seed}")

# Seed all RNGs
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)  # also seeds Hugging Face’s Trainer internals

🔢 Using random seed: 457209579


In [5]:
args = {
    "model_name_or_path":      "SolomonSLee/TINYdistillBert",
    "output_dir":              "./output/squad",
    "max_seq_length":          384,
    "doc_stride":              128,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size":  16,
    "learning_rate":           3e-5,
    "num_train_epochs":        3,
    "logging_steps":           500,
    "save_steps":              500,
    "seed":                    seed,
}

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    args["model_name_or_path"], use_fast=True
)

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
def prepare_features(examples):
    tokenized = tokenizer(
        examples['question'], examples['context'],
        truncation='only_second', max_length=384, stride=128,
        return_overflowing_tokens=True, return_offsets_mapping=True,
        padding='max_length'
    )
    sample_map = tokenized.pop('overflow_to_sample_mapping')
    offsets    = tokenized.pop('offset_mapping')
    starts, ends = [], []
    for i, offset in enumerate(offsets):
        input_ids  = tokenized['input_ids'][i]
        sample_idx = sample_map[i]
        answer     = examples['answers'][sample_idx]
        if not answer['answer_start']:
            starts.append(0); ends.append(0)
        else:
            start_char = answer['answer_start'][0]
            end_char   = start_char + len(answer['text'][0])
            seq_ids    = tokenized.sequence_ids(i)
            tok_start  = next(j for j, sid in enumerate(seq_ids) if sid == 1)
            tok_end    = len(input_ids) - 1 - next(j for j, sid in enumerate(reversed(seq_ids)) if sid == 1)
            if not (offset[tok_start][0] <= start_char and offset[tok_end][1] >= end_char):
                starts.append(0); ends.append(0)
            else:
                while tok_start < len(offset) and offset[tok_start][0] <= start_char:
                    tok_start += 1
                starts.append(tok_start - 1)
                while offset[tok_end][1] >= end_char:
                    tok_end -= 1
                ends.append(tok_end + 1)
    tokenized['start_positions'] = starts
    tokenized['end_positions']   = ends
    return tokenized

In [ ]:
tokenized_train = raw_train.map(prepare_features, batched=True, remove_columns=raw_train.column_names)
tokenized_eval  = raw_eval.map(prepare_features, batched=True, remove_columns=raw_eval.column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForQuestionAnswering.from_pretrained(
    args["model_name_or_path"]
)

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at SolomonSLee/TINYdistillBert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir=args["output_dir"],
    eval_strategy="no",
    learning_rate=args["learning_rate"],
    per_device_train_batch_size=args["per_device_train_batch_size"],
    per_device_eval_batch_size=args["per_device_eval_batch_size"],
    num_train_epochs=args["num_train_epochs"],
    logging_steps=args["logging_steps"],
    seed=args["seed"],
    overwrite_output_dir=True,
    remove_unused_columns = False,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

<ipython-input-10-d1dee311ce66>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,4.600000
1000,4.388200
1500,4.276500
2000,4.145600
2500,4.086400
3000,4.031100
3500,3.999400
4000,3.947500
4500,3.907100
5000,3.874700


TrainOutput(global_step=16599, training_loss=3.756464654088372, metrics={'train_runtime': 9640.8861, 'train_samples_per_second': 27.546, 'train_steps_per_second': 1.722, 'total_flos': 2.602335381127373e+16, 'train_loss': 3.756464654088372, 'epoch': 3.0})

In [ ]:
evaluator   = evaluate.load('squad')
qa_pipeline = pipeline('question-answering', model=model, tokenizer=tokenizer)
predictions = []
references  = []
for ex in raw_eval:
    out = qa_pipeline(question=ex['question'], context=ex['context'])
    predictions.append({'id': str(ex['id']), 'prediction_text': out['answer']})
    references.append({'id': str(ex['id']), 'answers': ex['answers']})
metrics = evaluator.compute(predictions=predictions, references=references)
print(metrics)

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'exact_match': 10.359508041627247, 'f1': 18.576486267938748}
